In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_csv = pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
print(data_csv['age'].unique()) #1~116
print(data_csv['ethnicity'].unique()) #0~4
data_csv.head()

In [ ]:
def age_cvt(age) :
    return int(age/10)

data_csv['age'] = data_csv['age'].apply(lambda x : age_cvt(x))
data_csv.head()

In [ ]:
import matplotlib.pyplot as plt

data_csv['pixels'] = data_csv['pixels'].apply(lambda x : np.array(x.split(), dtype="float32"))
data_csv['pixels'] = data_csv['pixels'] / 255.0

plt.imshow(data_csv['pixels'][0].reshape(48, 48)) #2304 = 48*48
plt.show()

In [ ]:
test_list = [4, 504, 1004, 2504, 5004, 6904, 17894, 18644, 20004, 23014]

print(len(data_csv))

test_csv = pd.DataFrame()
for i in test_list :
    test_csv = test_csv.append(data_csv.iloc[i])
    data_csv = data_csv.drop(i)

print(len(data_csv))
test_csv.head()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import cv2

class dataset(Dataset) :
    def __init__(self, age_label, eth_label, gen_label, img) :
        self.age_label = np.array(age_label)
        self.eth_label = np.array(eth_label)
        self.gen_label = np.array(gen_label)
        self.img = np.array(img)
        
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])
        
    def __len__(self) :
        return len(self.img)
    
    def __getitem__(self, index) :
        age_label = self.age_label[index]
        eth_label = self.eth_label[index]
        gen_label = self.gen_label[index]
        
        img = self.img[index].reshape(48, 48, 1)
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        img = self.transform(img)
        
        return img, age_label, eth_label, gen_label
    
train_data = dataset(data_csv['age'][:], data_csv['ethnicity'][:], 
                     data_csv['gender'][:], data_csv['pixels'][:])
train_data = DataLoader(train_data, batch_size = 128, shuffle = True)

In [ ]:
age_features = len(data_csv['age'].unique())
eth_features = len(data_csv['ethnicity'].unique())
gen_features = len(data_csv['gender'].unique())

print(age_features, eth_features, gen_features)

In [ ]:
!pip install timm

In [ ]:
!pip install torchsummary

In [ ]:
import torch.nn as nn
import timm
from timm.models.layers.classifier import ClassifierHead
from torchsummary import summary

class MultiLabelCNN(nn.Module) :
    def __init__(self, age_features, ethnicity_features, gender_features) :
        super().__init__()   
        '''
        self.model = timm.create_model('dm_nfnet_f0', pretrained = False)
        n_features = self.model.num_features
        self.age_classifier = ClassifierHead(n_features, age_features)
        self.eth_classifier = ClassifierHead(n_features, ethnicity_features)
        self.gen_classifier = ClassifierHead(n_features, gender_features)
        '''
        self.cnnModel = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #24
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #12
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #6
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #3
            #nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1,1)) #flatten
        )
        
        self.dnnModel = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
        )
        
        self.age_classifier = nn.Linear(32, age_features)
        self.eth_classifier = nn.Linear(32, ethnicity_features)
        self.gen_classifier = nn.Linear(32, gender_features)     
    def forward(self, x) :
        '''
        output = self.model.forward_features(x)
        age = self.age_classifier(output)
        eth = self.eth_classifier(output)
        gen = self.gen_classifier(output)
        '''
        output = self.cnnModel(x)
        output = output.squeeze()
        output = self.dnnModel(output)
        
        age = self.age_classifier(output)
        eth = self.eth_classifier(output)
        gen = self.gen_classifier(output)
        return age, eth, gen
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiLabelCNN(age_features, eth_features, gen_features).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

def custom_loss (output, target) :
    age_loss = nn.CrossEntropyLoss().to(device)(output[0], target[0])
    eth_loss =  nn.CrossEntropyLoss().to(device)(output[1], target[1])
    gen_loss =  nn.CrossEntropyLoss().to(device)(output[2], target[2])
    
    return age_loss + eth_loss + gen_loss, age_loss.item(), eth_loss.item(), gen_loss.item()

summary(model, input_size=(1, 48, 48))

In [ ]:
from tqdm.notebook import tqdm

epoch_num = 100
model.train()

for epoch in range(epoch_num) :
    epoch_loss = 0
    epoch_acc = 0
    for img, age, eth, gen in tqdm(train_data) :
        img = img.to(device)
        age = age.to(device)
        eth = eth.to(device)
        gen = gen.to(device)
        
        predict = model(img)
        
        loss = custom_loss(predict, (age, eth, gen))
        
        optimizer.zero_grad()
        loss[0].backward()
        optimizer.step()
        
        correct_prediction = 0
        for i, _ in enumerate(age) :
            if torch.argmax(predict[0][i]) == age[i] and \
            torch.argmax(predict[1][i]) == eth[i] and \
            torch.argmax(predict[2][i]) == gen[i] :
                correct_prediction += 1
                
        epoch_loss += loss[0]
        epoch_acc += correct_prediction / img.shape[0]
        
    epoch_loss = epoch_loss / len(train_data)
    epoch_acc = epoch_acc / len(train_data)
    print('Epoch : {}/{},   loss : {:.5f},    acc : {:.5f}'.format(epoch+1, epoch_num, epoch_loss, epoch_acc))
    
    if epoch_acc > 0.93 and epoch_loss < 0.2 :
        print('early stop')
        break

In [ ]:
model.eval()

test_data = dataset(test_csv['age'][:], test_csv['ethnicity'][:], \
                     test_csv['gender'][:], test_csv['pixels'][:])
test_data = DataLoader(test_data, batch_size = 1, shuffle = False)
print(len(test_data))

correct_prediction = 0
for img, age, eth, gen in test_data :
    img = img.to(device)
    age = age.to(device)
    eth = eth.to(device)
    gen = gen.to(device)
    
    predict = model(img)
    
    real = str(age[0].item()) + ' ' + str(eth[0].item()) + ' ' + str(gen[0].item())
    pred = str(torch.argmax(predict[0]).item()) + ' ' + \
    str(torch.argmax(predict[1]).item()) + ' ' + str(torch.argmax(predict[2]).item())
    
    img = img.reshape(1, 48, 48).to('cpu').numpy()
    plt.imshow(img.transpose(1, 2, 0))
    plt.title('real : ' + str(real) + ' / predict : ' + str(pred))
    plt.show()
    
    if age[0] == torch.argmax(predict[0]) and \
    eth[0] == torch.argmax(predict[1]) and \
    gen[0] == torch.argmax(predict[2]) : correct_prediction += 1

print('total_acc : ', str(correct_prediction / len(test_data)))